# Run evaluation script

In [ ]:
from evaluation import Evaluation

In [ ]:
evaluator = Evaluation('sample_query.csv')
df_combined = evaluator.get_full_eval()

In [4]:
df_combined.to_csv("eval_output.csv", index=False)

# Evaluation

In [2]:
import pandas as pd
from tqdm import tqdm
import re
from vectordb_retriever import VectorDbRetriever, GraphDbRetriever, Reranker
from weighted_vectordb_retriever import WeightedGraphDbRetriever, WeightedReranker
from dfs_vectordb_retriever import DFSRetriever
from classify_query import QueryClassifierAgent
from summary_and_output import SummaryAgent, OutputAgent
from source_router import SourceRouterAgent
from llm_as_judge import LLMAsJudge
from sentence_transformers import SentenceTransformer, util

c:\Users\Victoria Magdalena\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
# Load the chunks 
import json 
with open(r"ba_data_extraction\banking_act.json", "r", encoding="utf-8") as file:
    ba_chunks = json.load(file)

ba_dict = {}
for chunk in ba_chunks:
    ba_dict[chunk['id']] = chunk['text']

with open(r"mas_data_extraction\mas_chunks.json", "r", encoding="utf-8") as file:
    mas_chunks = json.load(file)

mas_dict = {}
for chunk in mas_chunks:
    mas_dict[chunk['id']] = chunk['text']

combined_dict = {**ba_dict, **mas_dict}

In [15]:
def get_text(id, dict=combined_dict):
    return dict[id]

model = SentenceTransformer('all-MiniLM-L6-v2')
def get_similarity_score(chunk1, chunk2):
    embeddings = model.encode([chunk1, chunk2])
    score = util.cos_sim(embeddings[0], embeddings[1]).item()
    return score

def get_ranking_score(retrieved):
    
    # Initialize total penalty score
    correct = sorted(retrieved, reverse=True)

    retrieved = [correct.index(i) for i in retrieved]
    correct = [correct.index(i) for i in correct]
    
    total_penalty = 0
    max_penalty = 0
    n = len(correct)
    
    # Compare each pair of chunks in the retrieved list
    for i in range(n):
        for j in range(i + 1, n):
            # Check if there is an inversion (higher value chunk before lower value chunk)
            if correct.index(retrieved[i]) > correct.index(retrieved[j]):
                # Calculate the penalty based on the difference in chunk values
                penalty = abs(retrieved[i] - retrieved[j])
                total_penalty += penalty  # Add penalty to total

    for i in range(n):
        for j in range(i + 1, n):
            # Maximum possible penalty when elements are completely reversed
            max_penalty += abs(correct[i] - correct[j])
        
    # Normalize the penalty score to be between 0 and 1
    normalized_penalty = total_penalty / max_penalty
    
    return 1 - normalized_penalty

def get_retriever_score(reference_ids, top_k_chunks):
    # Calculate position score
    position_scores = [] 
    # Using semantic similarity score 
    max_scores = []
    rank_scores = []
    
    # 'reference_retrieval':['ba-1970-2.1', 'ba-1970-4.2']
    if len(top_k_chunks) > 0:
        for ref_id in reference_ids:
            # Get position_scores 
            if ref_id in top_k_chunks:
                position = top_k_chunks.index(ref_id) 
            else:
                position = len(top_k_chunks)
            position_score = (len(top_k_chunks) - position) / len(top_k_chunks) if len(top_k_chunks) > 0 else 0
            position_scores.append(position_score)

            # Get semantic similarity score between the reference chunk and the retrieved chunk
            retrieved_chunks_score = [get_similarity_score(get_text(ref_id), get_text(chunk_id)) for chunk_id in top_k_chunks]
            print('Retrieved chunks similarity score:', retrieved_chunks_score)

            # Store max similarity score (range -1 to 1)
            max_score = max(retrieved_chunks_score) 
            max_scores.append(max_score)
            
            # Evaluate the ranking/order  
            rank_score = get_ranking_score(retrieved_chunks_score)
            rank_scores.append(rank_score)
    else:
        position_scores = [0]
        max_scores = [0]
        rank_scores = [0]

    print('Position scores:', position_scores)
    print('Max scores:', max_scores)
    print('Rank scores:', rank_scores)

    agg_position_score = max(position_scores)
    agg_max_score = max(max_scores)
    agg_rank_score = max(rank_scores)

    return agg_position_score, agg_max_score, agg_rank_score, position_scores, max_scores, rank_scores

def get_evaluation(query, correct_source, correct_type, reference_retrieval, reference_generation, n_hop, retriever_method):  
    user_query = query
    
    output = {}
    output['query'] = query
    print('Query:', query)

    # Determine data source 
    output['correct_source'] = correct_source
    router = SourceRouterAgent()
    data_source = router.get_source(user_query)
    print("Raw Data Source:", data_source) 

    # Cleaning data source
    if 'ba' in data_source.lower():
        data_source = 'ba'
    elif 'mas' in data_source.lower():
        data_source = 'mas'
    else:
        output['source'] = data_source
        output['is_source_correct'] = 0
        return output 

    output['source'] = data_source
    if data_source == correct_source:
        output['is_source_correct'] = 1
    else: 
        output['is_source_correct'] = 0

    # Classify the query
    output['correct_type'] = correct_type
    classifier_agent = QueryClassifierAgent()
    classification = classifier_agent.classify_query(user_query)
    print("Raw Classification:", classification)

    if 'factual' in classification.lower():
        classification = 'factual'
    elif 'reasoning' in classification.lower():
        classification = 'reasoning'
    else: 
        # assign misclassification as factual for simplicity
        classification = 'factual'
    
    output['type'] = classification
    if classification == correct_type: 
        output['is_type_correct'] = 1
    else: 
        output['is_type_correct'] = 0 
        
    # Retrieve top-k chunks from VectorDB 
    vectorretriever = VectorDbRetriever(top_k=10)
    top_k_chunks = vectorretriever.get_top_k_chunks(user_query, data_source)
    print("VectorDB Chunks:", top_k_chunks)
    agg_position_score, agg_max_score, agg_rank_score, position_scores, max_scores, rank_scores = get_retriever_score(reference_retrieval, top_k_chunks)
    
    output['reference_retrieval'] = reference_retrieval
    output['generated_retrieval'] = top_k_chunks
    output['retrieval_position_scores'] = position_scores
    output['retrieval_max_position_score'] = agg_position_score
    output['retrieval_best_similarity_scores'] = max_scores
    output['retrieval_max_similarity_score'] = agg_max_score
    output['retrieval_rank_similarity_scores'] = rank_scores
    output['retrieval_max_rank_similarity_score'] = agg_rank_score
    
    # GraphDB Retriever 
    output['graphdb_retrieval_method'] = retriever_method
    output['n_hops'] = n_hop
    if retriever_method == 'bfs':
        graphretriever = GraphDbRetriever(hops=n_hop)
        appended_chunks = graphretriever.get_appended_chunks(top_k_chunks, data_source)
        reranker = Reranker(top_k=5)
    elif retriever_method == 'weighted':
        graphretriever = WeightedGraphDbRetriever(hops=n_hop)
        appended_chunks = graphretriever.get_appended_chunks(top_k_chunks, data_source)
        reranker = WeightedReranker(top_k=5)
    elif retriever_method == 'dfs':
        graphretriever = DFSRetriever(path_length=n_hop)
        appended_chunks = graphretriever.run_DFS(user_query, top_k_chunks, data_source)
        reranker = Reranker(top_k=5)

    print('Appended chunks:', appended_chunks)
    # Rerank top_k appended chunks 
    reranked_chunks = reranker.rerank(user_query, appended_chunks)
    print('Reranked chunks:', reranked_chunks)
    print('Length Reranked chunks:', len(reranked_chunks))

    # Extrack reranked chunk ids
    reranked_ids = []
    for chunk in reranked_chunks:
        if retriever_method == 'weighted':
            id_pattern = re.search(r'^ID:\s(.*?)(?=\n)', chunk)
            chunk_id = id_pattern.group(1) if id_pattern else ''
            if chunk_id != '':
                reranked_ids.append(chunk_id)
        else:
            id_pattern = re.match(r"\(([^)]+)\)", chunk)
            chunk_id = id_pattern.group(1) if id_pattern else ''
            if chunk_id != '':
                reranked_ids.append(chunk_id)

    print('Reranker ids:', reranked_ids)
    
    agg_position_score, agg_max_score, agg_rank_score, position_scores, max_scores, rank_scores = get_retriever_score(reference_retrieval, reranked_ids)
    output['reranker_generated_retrieval'] = reranked_ids
    output['reranker_position_scores'] = position_scores
    output['reranker_max_position_score'] = agg_position_score
    output['reranker_best_similarity_scores'] = max_scores
    output['reranker_max_similarity_score'] = agg_max_score
    output['reranker_rank_similarity_scores'] = rank_scores
    output['reranker_max_rank_similarity_score'] = agg_rank_score
    
    # Summary reranked appended chunks 
    summary_agent = SummaryAgent()
    judge = LLMAsJudge()
    summarized_chunks = []
    summary_scores = []
    for chunk in reranked_chunks:
        id_pattern = re.match(r"\(([^)]+)\)", chunk)

        if id_pattern:
            chunk_id = id_pattern.group(1) 
            if data_source == 'ba':
                chunk_id = chunk_id.replace('ba', 'Banking Act')
            elif data_source == 'mas':
                chunk_id = chunk_id.replace('mas', 'MAS')

            before, sep, after = chunk_id.rpartition('-')
            chunk_id = before + ', ' + after
            chunk_id = chunk_id.replace('-', ' ').title()
        else: 
            chunk_id = ''    

        summary = summary_agent.summarize_text(chunk, user_query)
        summary = summary.replace('\n', ' ')
        summarized_chunks.append(f'{chunk_id}: {summary}')

        # Evaluate summary using LLM As Judge
        summary_score = judge.judge_summary(user_query, chunk, summary)
        summary_score = re.findall(r'\d+', summary_score)
        summary_score = int(summary_score[0]) if summary_score else 0
        summary_scores.append(summary_score)
    final_chunks = '\n\n'.join(summarized_chunks)
    output['summary_score'] = sum(summary_scores)/len(summary_scores)

    # Get final answer
    output_agent = OutputAgent()
    final_answer = output_agent.output_response(data_source, classification, user_query, summarized_chunks=final_chunks)
    output['reference_answer'] = reference_generation
    output['generated_answer'] = final_answer

    # Evaluate output agent with LLM As Judge 
    answer_score = judge.judge_answer(user_query, reference_generation, final_answer)
    answer_score = re.findall(r'\d+', answer_score)
    answer_score = int(answer_score[0]) if answer_score else 0
    output['answer_score'] = answer_score
    print('Answer score:', answer_score)

    print('---------------------------------------------------\n')
    return output

In [6]:
df_queries = pd.read_csv('sample_query.csv', index_col=0)
df_queries

,query,correct_source,correct_type,reference_retrieval,reference_generation
0,What is the minimum capital requirement for a ...,ba,factual,ba-1970-9.1,A foreign bank must have head office capital f...


In [7]:
# Check if all reference retrieval chunks are matched
for id, row in df_queries.iterrows():
    id_list = [row['reference_retrieval']] if ',' not in row['reference_retrieval'] else row['reference_retrieval'].split(', ')
    for id in id_list:
        cleaned_id = id.replace('-', ' ').replace('ba', 'Banking Act').replace('.', '(') + ')'
        print(f"{cleaned_id}:\n{get_text(id)}\n")

Banking Act 1970 9(1):
Part 3. Licensing of banks
Section 9. Minimum capital requirements
9(1) Subject to this Act, a company must not be granted a bank licence unless — (a) where the company intends to carry on banking business in Singapore as a bank incorporated in Singapore, its paid‑up capital is at least $1,500 million or such other amount as may be prescribed, and its capital funds are at least that amount; or (b) where the company intends to carry on banking business in Singapore through a branch or office located in Singapore, its head office capital funds are at least the equivalent of $200 million. [5/2016; 1/2020]



In [ ]:
# Get result for BFS first 
bfs_queries_evaluation = []
bfs_retriever_method = 'bfs'
for id, row in df_queries.iterrows():
    for n_hop in [1, 3, 5]:
        query = row['query']
        correct_source = row['correct_source']
        correct_type = row['correct_type']
        reference_retrieval = [row['reference_retrieval']] if ',' not in row['reference_retrieval'] else row['reference_retrieval'].split(', ')
        reference_generation = row['reference_generation']
        evaluation = get_evaluation(query, correct_source, correct_type, reference_retrieval, reference_generation, n_hop, bfs_retriever_method)
        bfs_queries_evaluation.append(evaluation)

df_bfs = pd.DataFrame(bfs_queries_evaluation)

In [ ]:
# df_bfs.to_csv('df_bfs.csv')

In [8]:
# Find the best hop
df_hop_score = df_bfs.groupby('n_hops')['answer_score'].mean().reset_index()
bfs_best_hop = int(df_hop_score.sort_values(by=['answer_score', 'n_hops'], ascending=[False, True]).iloc[0]['n_hops'])
print('BFS best hop:', bfs_best_hop)

BFS best hop: 3


In [9]:
df_hop_score

,n_hops,answer_score
0,1,4.0
1,3,4.7
2,5,4.5


In [10]:
# Get result for DFS
dfs_queries_evaluation = []
for id, row in df_queries.iterrows():
    query = row['query']
    correct_source = row['correct_source']
    correct_type = row['correct_type']
    reference_retrieval = [row['reference_retrieval']] if ',' not in row['reference_retrieval'] else row['reference_retrieval'].split(', ')
    reference_generation = row['reference_generation']
    evaluation = get_evaluation(query, correct_source, correct_type, reference_retrieval, reference_generation, n_hop=bfs_best_hop, retriever_method='dfs')
    dfs_queries_evaluation.append(evaluation)

df_dfs = pd.DataFrame(dfs_queries_evaluation)

Query: What is the minimum capital requirement for a foreign bank that wants to operate in Singapore through a branch?
Raw Data Source: ba
Raw Classification: Factual
VectorDB Chunks: ['ba-1970-9.1', 'ba-1970-55t.1', 'ba-1970-12.3', 'ba-1970-9.3a', 'ba-1970-40.1', 'ba-1970-38.1', 'ba-1970-10.1', 'ba-1970-31.1a', 'ba-1970-10.2', 'ba-1970-9a.1']
Retrieved chunks similarity score: [0.9999998807907104, 0.8659762740135193, 0.7440372109413147, 0.7704546451568604, 0.7480430006980896, 0.6859292984008789, 0.7692022919654846, 0.8021577596664429, 0.7345101833343506, 0.8171002864837646]
Position scores: [1.0]
Max scores: [0.9999998807907104]
Rank scores: [0.6606060606060606]


Getting next chunk in path: 100%|██████████| 3/3 [05:24<00:00, 108.02s/it]


Root: ba-1970-9.1//Path_id: ['ba-1970-9.1', 'ba-1970-10.1', 'ba-1970-9.3a', 'ba-1970-9a.1']//Text: ['9(1) Subject to this Act, a company must not be granted a bank licence unless — (a) where the company intends to carry on banking business in Singapore as a bank incorporated in Singapore, its paid‑up capital is at least $1,500 million or such other amount as may be prescribed, and its capital funds are at least that amount; or (b) where the company intends to carry on banking business in Singapore through a branch or office located in Singapore, its head office capital funds are at least the equivalent of $200 million. [5/2016; 1/2020]', '10(1) The Authority may, by written notice, require any bank in Singapore or class of banks in Singapore to maintain capital funds in Singapore of such amount (not being less than the minimum prescribed in section 9 or 9A, as the case may be) and in such manner as the Authority considers appropriate, having regard to the risks arising from the activit

Reranking chunks: 100%|██████████| 10/10 [00:13<00:00,  1.35s/it]


['(ba-1970-9.3a) 9(3A) A bank must at all times maintain — (a) if it is a bank incorporated in Singapore, capital funds of at least the amount mentioned in subsection (1)(a); or (b) if it is a bank incorporated outside Singapore, head office capital funds of at least the equivalent of the amount mentioned in subsection (1)(b). [5/2016]   (ba-1970-9.1) 9(1) Subject to this Act, a company must not be granted a bank licence unless — (a) where the company intends to carry on banking business in Singapore as a bank incorporated in Singapore, its paid‑up capital is at least $1,500 million or such other amount as may be prescribed, and its capital funds are at least that amount; or (b) where the company intends to carry on banking business in Singapore through a branch or office located in Singapore, its head office capital funds are at least the equivalent of $200 million. [5/2016; 1/2020]   (ba-1970-10.1) 10(1) The Authority may, by written notice, require any bank in Singapore or class of 

...

['(ba-1970-7.7) 7(7) Any bank which fails to comply with any of the conditions of its bank licence shall be guilty of an offence and shall be liable on conviction to a fine not exceeding $100,000 and, in the case of a continuing offence, to a further fine of $10,000 for every day during which the offence continues after conviction. [1/2020]   (ba-1970-2.1) 2(1) In this Act, unless the context otherwise requires — “agreement” means an agreement whether formal or informal and whether express or implied; “Authority” means the Monetary Authority of Singapore established under the Monetary Authority of Singapore Act 1970; “bank” means any company which holds a valid bank licence under section 7 or 79; “bank in Singapore” means — (a) a bank incorporated in Singapore; or (b) in the case of a bank incorporated outside Singapore, the branches and offices of the bank located within Singapore; “bank incorporated outside Singapore” means a bank incorporated, formed or established outside Singapore

Getting next chunk in path: 100%|██████████| 3/3 [04:08<00:00, 82.74s/it] 


Root: ba-1970-9a.2//Path_id: ['ba-1970-9a.2', 'ba-1970-9a.6a', 'ba-1970-9a.6', 'ba-1970-7.3']//Text: ['9A(2) A bank which is a qualifying subsidiary must not reduce its paid‑up capital, or purchase or otherwise acquire shares issued by the bank if such shares are to be held as treasury shares, without the approval of the Authority.', '9A(6A) Any bank which fails to comply with — (a) subsection (2), (3), (3A) or (5); or (b) any restriction or suspension imposed by the Authority, or any direction of the Authority, under subsection (6), shall be guilty of an offence and shall be liable on conviction to a fine not exceeding $250,000 and, in the case of a continuing offence, to a further fine not exceeding $25,000 for every day or part of a day during which the offence continues after conviction. [5/2016]', '9A(6) Where a bank fails to comply with any provision of this section or any condition imposed by the Authority under this section, the Authority may, without affecting subsection (6A),

Reranking chunks: 100%|██████████| 10/10 [00:10<00:00,  1.05s/it]


['(ba-1970-9.3) 9(3) A bank incorporated in Singapore must not reduce its paid‑up capital, or purchase or otherwise acquire shares issued by the bank if such shares are to be held as treasury shares, without the approval of the Authority.   (ba-1970-9a.1) 9A(1) Despite section 9, a company incorporated in Singapore which is a qualifying subsidiary may be granted a bank licence under section 7 if its paid‑up capital is at least $100 million. [5/2016; 1/2020]   (ba-1970-7.5) 7(5) The Authority must, prior to any action under subsection (4) — (a) notify its intention to take that action to the bank concerned; and (b) give the bank an opportunity to submit reasons why the conditions of its bank licence should not be so varied or revoked. [1/2020]   (ba-1970-2.1) 2(1) In this Act, unless the context otherwise requires — “agreement” means an agreement whether formal or informal and whether express or implied; “Authority” means the Monetary Authority of Singapore established under the Monetar

In [ ]:
# df_dfs.to_csv('df_dfs.csv')

In [12]:
# Get result for weighted
weighted_queries_evaluation = []
for id, row in df_queries.iterrows():
    query = row['query']
    correct_source = row['correct_source']
    correct_type = row['correct_type']
    reference_retrieval = [row['reference_retrieval']] if ',' not in row['reference_retrieval'] else row['reference_retrieval'].split(', ')
    reference_generation = row['reference_generation']
    evaluation = get_evaluation(query, correct_source, correct_type, reference_retrieval, reference_generation, n_hop=bfs_best_hop, retriever_method='weighted')
    weighted_queries_evaluation.append(evaluation)

df_weighted = pd.DataFrame(weighted_queries_evaluation)

Query: What is the minimum capital requirement for a foreign bank that wants to operate in Singapore through a branch?
Raw Data Source: ba
Raw Classification: Factual
VectorDB Chunks: ['ba-1970-9.1', 'ba-1970-55t.1', 'ba-1970-12.3', 'ba-1970-9.3a', 'ba-1970-40.1', 'ba-1970-38.1', 'ba-1970-10.1', 'ba-1970-31.1a', 'ba-1970-10.2', 'ba-1970-9a.1']
Retrieved chunks similarity score: [0.9999998807907104, 0.8659762740135193, 0.7440372109413147, 0.7704546451568604, 0.7480430006980896, 0.6859292984008789, 0.7692022919654846, 0.8021577596664429, 0.7345101833343506, 0.8171002864837646]
Position scores: [1.0]
Max scores: [0.9999998807907104]
Rank scores: [0.6606060606060606]


c:\Users\Victoria Magdalena\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Victoria Magdalena\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Document: ID: ba-1970-9a.1
9A(1) Despite section 9, a company incorporated in Singapore which is a qualifying ...
Combined Score: 0.6816372871398926
--------------------------------------------------
Document: ID: ba-1970-9.1
9(1) Subject to this Act, a company must not be granted a bank licence unless — (a) ...
Combined Score: 0.6503081321716309
--------------------------------------------------
Document: ID: ba-1970-31.1a
31(1A) A bank incorporated outside Singapore must not, through a branch or office ...
Combined Score: 0.6425287127494812
--------------------------------------------------
Document: ID: ba-1970-10.1
10(1) The Authority may, by written notice, require any bank in Singapore or class ...
Combined Score: 0.6271066069602966
--------------------------------------------------
Document: ID: ba-1970-40.1
40(1) The Authority may, from time to time, by written notice to any bank in Singap...
Combined Score: 0.6228955388069153
--------------------------------------------------


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n                       MATCH (n {id: $chunk_id})\n                       OPTIONAL MATCH (n)-[:REFERRED_BY*1..3]-(m)\n                       WITH n, COLLECT(m) AS connected_nodes\n                       RETURN \n                           CASE WHEN SIZE(connected_nodes) = 0 THEN (n)\n                           ELSE connected_nodes\n                        END AS result\n                '


Document: ID: ba-1970-26.2
26(2) Every bank must send to the Authority — (a) not later than 3 months after the...
Combined Score: 0.651705265045166
--------------------------------------------------
Document: ID: ba-1970-27.4
27(4) A statement under subsection (1)(a) — (a) must be prepared within the period ...
Combined Score: 0.6478741765022278
--------------------------------------------------
Document: ID: ba-1970-25.4
25(4) The Authority may, by written notice, require a bank to publish in addition t...
Combined Score: 0.6290768980979919
--------------------------------------------------
Document: ID: ba-1970-25.2
25(2) Every bank must, within 5 months after the close of each financial year or wi...
Combined Score: 0.5842620730400085
--------------------------------------------------
Document: ID: ba-1970-25.1a
25(1A) The Authority may, by regulations made under section 78(1), require a bank ...
Combined Score: 0.5376352667808533
--------------------------------------------------
D

In [ ]:
# df_weighted.to_csv('df_weighted.csv')

In [ ]:
df_combined = pd.concat([df_bfs, df_dfs, df_weighted])
df_combined = df_combined.reset_index(drop=True)
# df_combined.to_csv('df_combined.csv')

,query,correct_source,source,is_source_correct,correct_type,type,is_type_correct,reference_retrieval,generated_retrieval,retrieval_position_scores,...,reranker_position_scores,reranker_max_position_score,reranker_best_similarity_scores,reranker_max_similarity_score,reranker_rank_similarity_scores,reranker_max_rank_similarity_score,summary_score,reference_answer,generated_answer,answer_score
0,What is the minimum capital requirement for a ...,ba,ba,1,factual,factual,1,['ba-1970-9.1'],"['ba-1970-9.1', 'ba-1970-55t.1', 'ba-1970-12.3...",[1.0],...,[1.0],1.0,[0.9999998807907104],1.000000,[0.65],0.65,4.8,A foreign bank must have head office capital f...,The minimum capital requirement for a foreign ...,5
1,What is the minimum capital requirement for a ...,ba,ba,1,factual,factual,1,['ba-1970-9.1'],"['ba-1970-9.1', 'ba-1970-55t.1', 'ba-1970-12.3...",[1.0],...,[1.0],1.0,[0.9999998807907104],1.000000,[0.85],0.85,5.0,A foreign bank must have head office capital f...,The minimum capital requirement for a foreign ...,5
2,What is the minimum capital requirement for a ...,ba,ba,1,factual,factual,1,['ba-1970-9.1'],"['ba-1970-9.1', 'ba-1970-55t.1', 'ba-1970-12.3...",[1.0],...,[1.0],1.0,[0.9999998807907104],1.000000,[0.85],0.85,5.0,A foreign bank must have head office capital f...,The minimum capital requirement for a foreign ...,5
3,What financial statements and reports must a b...,ba,ba,1,factual,factual,1,"['ba-1970-25.2', 'ba-1970-26.2']","['ba-1970-25.2', 'ba-1970-26.1', 'ba-1970-26.2...","[1.0, 0.8]",...,"[1.0, 0.4]",1.0,"[1.0, 1.0000001192092896]",1.000000,"[0.8, 0.6]",0.80,5.0,"Under the Banking Act 1970, a bank must both p...",Every bank is required to submit its latest au...,4
4,What financial statements and reports must a b...,ba,ba,1,factual,factual,1,"['ba-1970-25.2', 'ba-1970-26.2']","['ba-1970-25.2', 'ba-1970-26.1', 'ba-1970-26.2...","[1.0, 0.8]",...,"[0.6, 0.4]",0.6,"[1.0, 1.0000001192092896]",1.000000,"[0.8, 0.5]",0.80,5.0,"Under the Banking Act 1970, a bank must both p...",A bank must submit the following financial sta...,5
5,What financial statements and reports must a b...,ba,ba,1,factual,factual,1,"['ba-1970-25.2', 'ba-1970-26.2']","['ba-1970-25.2', 'ba-1970-26.1', 'ba-1970-26.2...","[1.0, 0.8]",...,"[0.6, 0.4]",0.6,"[1.0, 1.0000001192092896]",1.000000,"[0.8, 0.5]",0.80,5.0,"Under the Banking Act 1970, a bank must both p...",Every bank is required to submit the following...,5
6,Is a bank allowed to appeal a licence revocati...,ba,ba,1,factual,factual,1,['ba-1970-20.4'],"['ba-1970-11a', 'ba-1970-20.4', 'ba-1970-55za....",[0.9],...,[0.4],0.4,[1.0000001192092896],1.000000,[0.19999999999999996],0.20,4.8,"Yes, a bank is allowed to appeal a licence rev...","Yes, a bank is allowed to appeal a licence rev...",4
7,Is a bank allowed to appeal a licence revocati...,ba,ba,1,factual,factual,1,['ba-1970-20.4'],"['ba-1970-11a', 'ba-1970-20.4', 'ba-1970-55za....",[0.9],...,[0.8],0.8,[1.0000001192092896],1.000000,[0.75],0.75,5.0,"Yes, a bank is allowed to appeal a licence rev...","Yes, a bank is allowed to appeal a licence rev...",5
8,Is a bank allowed to appeal a licence revocati...,ba,ba,1,factual,factual,1,['ba-1970-20.4'],"['ba-1970-11a', 'ba-1970-20.4', 'ba-1970-55za....",[0.9],...,[0.8],0.8,[1.0000001192092896],1.000000,[0.75],0.75,5.0,"Yes, a bank is allowed to appeal a licence rev...","Yes, a bank has the right to appeal a licence ...",5
9,What are the consequences for a bank that open...,ba,ba,1,factual,factual,1,['ba-1970-12.7'],"['ba-1970-12.1', 'ba-1970-9.5a', 'ba-1970-10.5...",[0.0],...,[0.0],0.0,[0.9465527534484863],0.946553,[0.25],0.25,4.4,A bank that opens a new place of business or c...,A bank that opens a new place of business or c...,5


In [15]:
df_method_score = df_combined[df_combined['n_hops'] == bfs_best_hop].groupby('graphdb_retrieval_method')['answer_score'].mean().reset_index()
best_method = df_method_score.sort_values(by=['answer_score'], ascending=[False]).iloc[0]['graphdb_retrieval_method']
print('Best retrieval method:', best_method)

Best retrieval method: bfs


In [16]:
df_method_score

,graphdb_retrieval_method,answer_score
0,bfs,4.7
1,dfs,3.9
2,weighted,3.8
